In [2]:
from astropy.io import fits
import numpy as np
from skimage import exposure
import os
import cv2
import pandas as pd
from ultralytics import YOLO

def brighten_image(image):
    brightened_image = np.clip(image * 1.5, 0, 65535).astype(np.uint16)
    
    equalized_image = exposure.equalize_hist(brightened_image)
    
    gamma_corrected_image = exposure.adjust_gamma(equalized_image, gamma=2.0) 
    
    return gamma_corrected_image

def process_fits_images(input_dir):
    processed_images = {}
    
    for filename in os.listdir(input_dir):
        if filename.endswith(".fits"):
            fits_path = os.path.join(input_dir, filename)
            hdulist = fits.open(fits_path)
            img = hdulist[0].data
            hdulist.close()

            brightened_img = brighten_image(img)

            image_8bit = (brightened_img / np.max(brightened_img) * 255).astype(np.uint8)

            image_rgb = cv2.cvtColor(image_8bit, cv2.COLOR_GRAY2RGB)

            processed_images[filename] = image_rgb
    
    return processed_images

def detect_streak(processed_images, model_path, output_folder):
    model = YOLO(model_path)
    results_list = []

    for filename, image in processed_images.items():
        results = model.predict(source=image, save=True)

        object_count = 0
        for result in results:
            for box in result.boxes:
                object_count += 1
                x_center, y_center, width, height = box.xywh[0].tolist()
                x1 = int(x_center - width / 2)
                y1 = int(y_center - height / 2)
                x2 = int(x_center + width / 2)
                y2 = int(y_center + height / 2)

                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                cv2.circle(image, (x1, y1), 5, (0, 255, 0), -1)
                cv2.circle(image, (x2, y2), 5, (0, 255, 0), -1)
                cv2.circle(image, (x1, y2), 5, (0, 255, 0), -1)
                cv2.circle(image, (x2, y1), 5, (0, 255, 0), -1)
                cv2.circle(image, (center_x, center_y), 5, (255, 0, 0), -1)

                cv2.putText(image, f"({x1}, {y1})", (x1 + 10, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                cv2.putText(image, f"({x2}, {y2})", (x2 + 10, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                cv2.putText(image, f"({x1}, {y2})", (x1 + 10, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                cv2.putText(image, f"({x2}, {y1})", (x2 + 10, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                cv2.putText(image, f"Center: ({center_x}, {center_y})", (center_x + 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                results_list.append([filename, object_count, x1, y1, x2, y2, center_x, center_y])

        if object_count == 0:
            results_list.append([filename, object_count, "No detection", None, None, None, None, None])

        output_image_path = os.path.join(output_folder, filename.replace(".fits", ".png"))
        cv2.imwrite(output_image_path, image)

    df = pd.DataFrame(results_list, columns=["File name", "Number of objects", "x1", "y1", "x2", "y2", "x0", "y0"])
    return df

input_directory = r"C:\Users\Admin\Desktop\BHOBH_WORKS_2025\GISTDA_WORKS_2025\Tracking\Datasets\Raw\Not_all\streak_like_feature\56123"
output_directory = r"C:\Users\Admin\Desktop\result\Fit_yolo"
model_path = r"C:\Users\Admin\Downloads\best.pt"

processed_images = process_fits_images(input_directory)

df_results = detect_streak(processed_images, model_path, output_directory)

df_results.to_csv(r"C:\Users\Admin\Desktop\result\Fit_yolo\results.csv", index=False)

print("Successfully completed.")



0: 640x640 1 Object, 1404.5ms
Speed: 31.9ms preprocess, 1404.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3

0: 640x640 1 Object, 2211.5ms
Speed: 66.1ms preprocess, 2211.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3

0: 640x640 1 Object, 1605.6ms
Speed: 64.2ms preprocess, 1605.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3

0: 640x640 1 Object, 2072.0ms
Speed: 32.4ms preprocess, 2072.0ms inference, 10.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3

0: 640x640 1 Object, 1801.8ms
Speed: 41.3ms preprocess, 1801.8ms inference, 6.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict3

0: 640x640 1 Object, 1506.7ms
Speed: 31.5ms preprocess, 1506.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\d